In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# ============================================================================
# STEP 0: CONFIG
# ============================================================================
print("="*80)
print("FARM BOY - GEOGRAPHIC CONTROL ANALYSIS")
print("="*80)

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("\nStep 1: Loading FSA coordinates and store data...")

ontario_fsa = pd.read_csv("ontario_fsa_with_coordinates.csv")  # clean lat/lon
farmboy_stores = pd.read_csv("farmboy_stores.csv")

print(f"  FSAs loaded: {len(ontario_fsa)}")
print(f"  Stores loaded: {len(farmboy_stores)}")

# Extract FSAs from store addresses
farmboy_stores['FSA'] = farmboy_stores['Address'].str.extract(r'([A-Z]\d[A-Z])')
store_fsas = farmboy_stores['FSA'].dropna().unique()
print(f"  Unique store FSAs: {len(store_fsas)}")

# ============================================================================
# STEP 2: DERIVED METRICS
# ============================================================================
print("\nStep 2: Creating derived metrics...")

ontario_fsa['education_rate'] = (
    ontario_fsa["Bachelor's degree or higher"] / ontario_fsa['Population, 2021']
) * 100

ontario_fsa['employment_rate'] = (
    ontario_fsa['Employed'] / ontario_fsa['Population, 2021']
) * 100

ontario_fsa['has_store'] = ontario_fsa['FSA'].isin(store_fsas)

# ============================================================================
# STEP 3: CALCULATE DISTANCE TO NEAREST STORE
# ============================================================================
print("\nStep 3: Calculating distance to nearest store...")

# Prepare store coordinates
store_coords = ontario_fsa[
    ontario_fsa['FSA'].isin(store_fsas) & ontario_fsa[['latitude', 'longitude']].notna().all(axis=1)
][['FSA', 'latitude', 'longitude']]

def distance_to_nearest_store(row):
    if row['has_store']:
        return 0
    if pd.isna(row['latitude']) or pd.isna(row['longitude']):
        return np.nan
    point = (row['latitude'], row['longitude'])
    min_dist = store_coords.apply(
        lambda s: geodesic(point, (s['latitude'], s['longitude'])).kilometers, axis=1
    ).min()
    return min_dist

ontario_fsa['distance_to_nearest_store'] = ontario_fsa.apply(distance_to_nearest_store, axis=1)

print("  ✓ Distance calculation complete")
print("Distance stats (km):")
print(ontario_fsa[ontario_fsa['distance_to_nearest_store'] > 0]['distance_to_nearest_store'].describe())

# ============================================================================
# STEP 4: REGION ASSIGNMENT
# ============================================================================
print("\nStep 4: Assigning regions based on FSA prefix...")

def assign_region(fsa):
    if pd.isna(fsa): return 'Other Ontario'
    first = fsa[0]
    second = fsa[1] if len(fsa) > 1 else ''
    if first == 'M': return 'Toronto'
    elif first == 'K' and second in ['1', '2']: return 'Ottawa'
    elif first == 'L' and second in ['4','5','6','7','9']: return 'GTA'
    else: return 'Other Ontario'

ontario_fsa['region'] = ontario_fsa['FSA'].apply(assign_region)

# Create region dummies
region_dummies = pd.get_dummies(ontario_fsa['region'], prefix='region', drop_first=False)
ontario_fsa = pd.concat([ontario_fsa, region_dummies], axis=1)

# ============================================================================
# STEP 5: REGRESSION ANALYSIS
# ============================================================================
print("\nStep 5: Running regression models...")

# Variables
demo_vars = ['education_rate', 'Median age of the population', 'Median total income of household in 2020 ($)', 'population_density']
X1 = ontario_fsa[demo_vars].dropna()
y = ontario_fsa.loc[X1.index, 'has_store'].astype(int)

# Distance + region variables
X2 = X1.copy()
X2['distance_to_nearest_store'] = ontario_fsa.loc[X1.index, 'distance_to_nearest_store']

region_cols_to_use = [c for c in region_dummies.columns if c != 'region_Other Ontario']
X3 = pd.concat([X1, ontario_fsa.loc[X1.index, region_cols_to_use]], axis=1)
X4 = pd.concat([X2, ontario_fsa.loc[X1.index, region_cols_to_use]], axis=1)

# Standardize
scaler1 = StandardScaler().fit(X1)
scaler2 = StandardScaler().fit(X2)
scaler3 = StandardScaler().fit(X3)
scaler4 = StandardScaler().fit(X4)

X1_scaled = scaler1.transform(X1)
X2_scaled = scaler2.transform(X2)
X3_scaled = scaler3.transform(X3)
X4_scaled = scaler4.transform(X4)

# Fit models
model1 = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced').fit(X1_scaled, y)
model2 = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced').fit(X2_scaled, y)
model3 = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced').fit(X3_scaled, y)
model4 = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced').fit(X4_scaled, y)

# AUC
auc1 = roc_auc_score(y, model1.predict_proba(X1_scaled)[:,1])
auc2 = roc_auc_score(y, model2.predict_proba(X2_scaled)[:,1])
auc3 = roc_auc_score(y, model3.predict_proba(X3_scaled)[:,1])
auc4 = roc_auc_score(y, model4.predict_proba(X4_scaled)[:,1])

print("\nModel AUCs:")
print(f"  Model 1 (Demo only): {auc1:.3f}")
print(f"  Model 2 (+Distance): {auc2:.3f}")
print(f"  Model 3 (+Region): {auc3:.3f}")
print(f"  Model 4 (Full): {auc4:.3f}")

# ============================================================================
# STEP 6: SAVE RESULTS
# ============================================================================
ontario_fsa.to_csv('ontario_fsa_with_full_geography.csv', index=False)
print("\n✓ Enhanced dataset saved: ontario_fsa_with_full_geography.csv")

FARM BOY - GEOGRAPHIC CONTROL ANALYSIS

Step 1: Loading FSA coordinates and store data...
  FSAs loaded: 520
  Stores loaded: 51
  Unique store FSAs: 50

Step 2: Creating derived metrics...

Step 3: Calculating distance to nearest store...
  ✓ Distance calculation complete
Distance stats (km):
count     469.000000
mean       83.300301
std       200.413970
min         0.619315
25%         4.825847
50%        11.534186
75%        53.411424
max      1311.995236
Name: distance_to_nearest_store, dtype: float64

Step 4: Assigning regions based on FSA prefix...

Step 5: Running regression models...

Model AUCs:
  Model 1 (Demo only): 0.740
  Model 2 (+Distance): 0.819
  Model 3 (+Region): 0.791
  Model 4 (Full): 0.852

✓ Enhanced dataset saved: ontario_fsa_with_full_geography.csv


In [2]:
# Compare coefficients
print("\n" + "-"*80)
print("COEFFICIENT COMPARISON - DEMOGRAPHIC VARIABLES")
print("-"*80)
print(f"{'Variable':<50} {'Model 1':<12} {'Model 2':<12} {'Model 3':<12} {'Model 4':<12}")
print("-"*80)

demo_vars = ['education_rate', 'Median age of the population', 
             'Median total income of household in 2020 ($)', 'population_density']

for var in demo_vars:
    idx1 = list(X1.columns).index(var)
    idx2 = list(X2.columns).index(var)
    idx3 = list(X3.columns).index(var)
    idx4 = list(X4.columns).index(var)
    
    coef1 = model1.coef_[0][idx1]
    coef2 = model2.coef_[0][idx2]
    coef3 = model3.coef_[0][idx3]
    coef4 = model4.coef_[0][idx4]
    
    print(f"{var:<50} {coef1:>11.3f} {coef2:>11.3f} {coef3:>11.3f} {coef4:>11.3f}")

# Show geographic controls
print("\n" + "-"*80)
print("GEOGRAPHIC CONTROL COEFFICIENTS")
print("-"*80)

idx_dist_2 = list(X2.columns).index('distance_to_nearest_store')
idx_dist_4 = list(X4.columns).index('distance_to_nearest_store')

print(f"Distance to Nearest Store:")
print(f"  Model 2 (Distance only):              {model2.coef_[0][idx_dist_2]:>8.3f}")
print(f"  Model 4 (Distance + Regions):         {model4.coef_[0][idx_dist_4]:>8.3f}")

print(f"\nRegion Coefficients (vs 'Other Ontario' reference):")
for col in region_cols_to_use:
    idx3 = list(X3.columns).index(col)
    idx4 = list(X4.columns).index(col)
    region_name = col.replace('region_', '')
    print(f"  {region_name:<30} Model 3: {model3.coef_[0][idx3]:>7.3f}   Model 4: {model4.coef_[0][idx4]:>7.3f}")


--------------------------------------------------------------------------------
COEFFICIENT COMPARISON - DEMOGRAPHIC VARIABLES
--------------------------------------------------------------------------------
Variable                                           Model 1      Model 2      Model 3      Model 4     
--------------------------------------------------------------------------------
education_rate                                           0.813       0.563       0.874       0.722
Median age of the population                            -0.364      -0.203      -0.246      -0.070
Median total income of household in 2020 ($)            -0.009      -0.042      -0.050      -0.083
population_density                                      -0.158      -0.127       0.020       0.033

--------------------------------------------------------------------------------
GEOGRAPHIC CONTROL COEFFICIENTS
--------------------------------------------------------------------------------
Distance to Nea

In [5]:
# ============================================================================
# STEP 7: INTERPRETATION OF RESULTS
# ============================================================================

print("\n" + "="*80)
print("KEY FINDINGS - INTERPRETATION")
print("="*80)

# Education coefficient
edu_1 = model1.coef_[0][list(X1.columns).index('education_rate')]
edu_4 = model4.coef_[0][list(X4.columns).index('education_rate')]
edu_change = ((edu_4 - edu_1) / abs(edu_1)) * 100

print(f"\n1. EDUCATION PREDICTOR:")
print(f"   Coefficient: {edu_1:.3f} (no controls) → {edu_4:.3f} (full controls)")
print(f"   Change: {edu_change:+.1f}%")

if abs(edu_4) >= abs(edu_1) * 0.7:
    print("   INTERPRETATION: Education remains strong after geographic controls")
    print("      → Education is a TRUE independent predictor")
    print("      → Not just a proxy for 'being in certain regions'")
    print("      → ACTION: Use education as PRIMARY criterion everywhere")
else:
    print("    INTERPRETATION: Education weakens substantially")
    print("      → Education effect partially explained by geography")
    print("      → Consider region-specific strategies")

# Distance effect
print(f"\n2. DISTANCE EFFECT:")
print(f"   Coefficient: {model2.coef_[0][idx_dist_2]:.3f}")

if abs(model2.coef_[0][idx_dist_2]) > 1.0:
    print("   VERY STRONG negative effect")
    print("      → Stores exhibit significant spatial clustering")
    print("      → Distance is major barrier to expansion")
    print("      → ACTION: Build regional clusters, avoid isolated stores")
else:
    print("   → Moderate distance effect")

# Model improvement
model_improvement = (auc2 - auc1) / auc1 * 100
print(f"\n3. MODEL IMPROVEMENT:")
print(f"   Adding distance control: +{model_improvement:.1f}% improvement")

if model_improvement > 20:
    print("   MASSIVE improvement")
    print("      → Geography is DOMINANT factor")
    print("      → Must account for location in site selection")
elif model_improvement > 10:
    print("   Substantial improvement")
    print("      → Geography matters significantly")
else:
    print("   → Modest improvement")

# ============================================================================
# STEP 8: REGIONAL SATURATION ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("REGIONAL SATURATION ANALYSIS")
print("="*80)

saturation = ontario_fsa.groupby('region').agg({
    'FSA': 'count',
    'has_store': 'sum',
    'Population, 2021': 'sum',
    'education_rate': 'mean',
    'distance_to_nearest_store': lambda x: x[x > 0].mean() if len(x[x > 0]) > 0 else 0
}).round(1)

saturation.columns = ['Total FSAs', 'Stores', 'Total Pop', 'Avg Edu %', 'Avg Dist (km)']
saturation['Store %'] = (saturation['Stores'] / saturation['Total FSAs'] * 100).round(1)
saturation['Stores per 100k'] = (saturation['Stores'] / saturation['Total Pop'] * 100000).round(2)

# Classify saturation
avg_penetration = saturation['Store %'].mean()
saturation['Saturation Level'] = saturation['Store %'].apply(
    lambda x: 'HIGH (Saturated)' if x > avg_penetration * 1.5 else
              'LOW (Opportunity)' if x < avg_penetration * 0.5 else
              'MEDIUM'
)

print("\nRegional Metrics:")
print(saturation.sort_values('Stores', ascending=False))

# ============================================================================
# STEP 9: IDENTIFY UNDERSERVED FSAs
# ============================================================================

print("\n" + "="*80)
print("UNDERSERVED HIGH-VALUE FSA IDENTIFICATION")
print("="*80)

# Criteria based on existing stores
with_stores = ontario_fsa[ontario_fsa['has_store']]

criteria = {
    'education_rate': (
        with_stores['education_rate'].quantile(0.25),
        with_stores['education_rate'].quantile(0.75)
    ),
    'Median age of the population': (
        with_stores['Median age of the population'].quantile(0.25),
        with_stores['Median age of the population'].quantile(0.75)
    ),
    'population_density': (
        with_stores['population_density'].quantile(0.25),
        with_stores['population_density'].quantile(0.75)
    ),
    'distance': (10, 50)  # 10-50km from nearest store (avoid cannibalization but not too far)
}

print(f"\nCriteria for underserved FSAs:")
print(f"  Education: {criteria['education_rate'][0]:.1f}% - {criteria['education_rate'][1]:.1f}%")
print(f"  Age: {criteria['Median age of the population'][0]:.1f} - {criteria['Median age of the population'][1]:.1f} years")
print(f"  Density: {criteria['population_density'][0]:.0f} - {criteria['population_density'][1]:.0f} per km²")
print(f"  Distance: {criteria['distance'][0]:.0f} - {criteria['distance'][1]:.0f} km from nearest store")

# Find underserved FSAs
underserved = ontario_fsa[
    (~ontario_fsa['has_store']) &
    (ontario_fsa['education_rate'] >= criteria['education_rate'][0]) &
    (ontario_fsa['education_rate'] <= criteria['education_rate'][1]) &
    (ontario_fsa['Median age of the population'] >= criteria['Median age of the population'][0]) &
    (ontario_fsa['Median age of the population'] <= criteria['Median age of the population'][1]) &
    (ontario_fsa['population_density'] >= criteria['population_density'][0]) &
    (ontario_fsa['population_density'] <= criteria['population_density'][1]) &
    (ontario_fsa['distance_to_nearest_store'] >= criteria['distance'][0]) &
    (ontario_fsa['distance_to_nearest_store'] <= criteria['distance'][1])
].copy()

print(f"\nTotal Underserved FSAs: {len(underserved)}")

if len(underserved) > 0:
    print("\nBy Region:")
    print(underserved.groupby('region').agg({
        'FSA': 'count',
        'Population, 2021': 'sum',
        'distance_to_nearest_store': 'mean'
    }).round(1))
    
    # Calculate opportunity score
    underserved['opportunity_score'] = (
        underserved['education_rate'] * 0.3 +
        (underserved['Population, 2021'] / 1000) * 0.3 +
        (45 - abs(40 - underserved['Median age of the population'])) * 2 * 0.2 +
        (50 - underserved['distance_to_nearest_store']) * 0.2  # Prefer closer (but not too close)
    )
    
    print("\nTop 15 Underserved Opportunities:")
    top_15 = underserved.nlargest(15, 'opportunity_score')[[
        'FSA', 'region', 'Population, 2021', 'education_rate',
        'Median age of the population', 'distance_to_nearest_store', 'opportunity_score'
    ]]
    print(top_15.to_string(index=False))

# ============================================================================
# STEP 10: SAVE RESULTS
# ============================================================================

print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Save enhanced dataset
ontario_fsa.to_csv('ontario_fsa_with_full_geography.csv', index=False)
print("Enhanced dataset saved: ontario_fsa_with_full_geography.csv")

# Save saturation
saturation.to_csv('regional_saturation_with_distance.csv')
print("Saturation metrics saved: regional_saturation_with_distance.csv")

# Save underserved
if len(underserved) > 0:
    underserved.sort_values('opportunity_score', ascending=False).to_csv(
        'underserved_fsas_ranked.csv', index=False
    )
    print("Underserved FSAs saved: underserved_fsas_ranked.csv")

# Save model comparison
model_results = pd.DataFrame({
    'Model': ['Demographics Only', '+ Distance', '+ Regions', '+ Distance + Regions (Full)'],
    'AUC': [auc1, auc2, auc3, auc4],
    'Change from Baseline (%)': [0, (auc2-auc1)*100, (auc3-auc1)*100, (auc4-auc1)*100]
})
model_results.to_csv('model_performance_comparison.csv', index=False)
print("Model results saved: model_performance_comparison.csv")

print("\n" + "="*80)
print("="*80)


KEY FINDINGS - INTERPRETATION

1. EDUCATION PREDICTOR:
   Coefficient: 0.813 (no controls) → 0.722 (full controls)
   Change: -11.2%
   INTERPRETATION: Education remains strong after geographic controls
      → Education is a TRUE independent predictor
      → Not just a proxy for 'being in certain regions'
      → ACTION: Use education as PRIMARY criterion everywhere

2. DISTANCE EFFECT:
   Coefficient: -5.100
   VERY STRONG negative effect
      → Stores exhibit significant spatial clustering
      → Distance is major barrier to expansion
      → ACTION: Build regional clusters, avoid isolated stores

3. MODEL IMPROVEMENT:
   Adding distance control: +10.7% improvement
   Substantial improvement
      → Geography matters significantly

REGIONAL SATURATION ANALYSIS

Regional Metrics:
               Total FSAs  Stores  Total Pop  Avg Edu %  Avg Dist (km)  \
region                                                                   
Other Ontario         299      16  7112550.0       13.4